In [1]:
import numpy as np
import torch
from giverny.turbulence_dataset import *
from giverny.turbulence_toolkit import *

In [ ]:
auth_token = 'your_auth_token_here'  # Replace with your actual authentication token
dataset_title = 'sabl2048high'
dataset = turb_dataset(dataset_title=dataset_title, output_path='', auth_token=auth_token)

In [3]:
print(dataset.metadata)

{'name': 'JHTDB', 'description': 'The original Johns Hopkins Turbulence Database', 'pickled_metadata_filepath': '/home/idies/workspace/turbulence-ceph/sciserver-turbulence/jhtdb_metadata/jhtdb_pickled', 'variables': [{'code': 'pressure', 'name': 'Pressure', 'description': None, 'component_codes': ['p'], 'cardinality': 1}, {'code': 'temperature', 'name': 'Temperature', 'description': None, 'component_codes': ['θ'], 'cardinality': 1}, {'code': 'soiltemperature', 'name': 'Soil Temperature', 'description': None, 'component_codes': ['θ-soil'], 'cardinality': 1}, {'code': 'sgsenergy', 'name': 'Subgrid-scale Energy', 'description': None, 'component_codes': ['e'], 'cardinality': 1}, {'code': 'sgsviscosity', 'name': 'Subgrid-scale Viscosity', 'description': None, 'component_codes': ['ν'], 'cardinality': 1}, {'code': 'density', 'name': 'Density', 'description': None, 'component_codes': ['ρ'], 'cardinality': 1}, {'code': 'velocity', 'name': 'Velocity', 'description': None, 'component_codes': ['ux

In [4]:
timesteps = list(range(1, 20))  # t=1 to 100
stride = 32
patch_size = 32

X = []
y = []

In [5]:
import numpy as np
import torch
import os
from tqdm import tqdm

# -------------------------------
# Configuration
# -------------------------------
patch_size = 32
stride = 32
timesteps = list(range(1, 10))

X, y = [], []

# -------------------------------
# Step 1: First Pass - Collect StdDev stats
# -------------------------------
std_vort_list, std_Tgrad_list, std_Pgrad_list = [], [], []

def compute_vorticity_2d(patch_v):
    u = patch_v[..., 0]
    v = patch_v[..., 1]
    dv_dx = np.gradient(v, axis=1)
    du_dy = np.gradient(u, axis=0)
    return dv_dx - du_dy

def compute_grad_magnitude(field):
    dx = np.gradient(field, axis=1)
    dy = np.gradient(field, axis=0)
    return np.sqrt(dx**2 + dy**2)

def compute_composite_metrics(patch_v, patch_T, patch_p):
    vort = compute_vorticity_2d(patch_v)
    grad_T = compute_grad_magnitude(patch_T)
    grad_P = compute_grad_magnitude(patch_p)
    return np.std(vort), np.std(grad_T), np.std(grad_P)

print("🔍 Collecting statistics for thresholding...")

for t in tqdm(timesteps):
    try:
        axes_ranges = np.array([[0, 2047], [0, 2047], [16, 16]], dtype=np.int32)
        strides_arr = np.array([1, 1, 1], dtype=np.int32)

        velocity_data = getCutout(dataset, 'velocity', t, axes_ranges, strides_arr)
        temperature_data = getCutout(dataset, 'temperature', t, axes_ranges, strides_arr)
        pressure_data = getCutout(dataset, 'pressure', t, axes_ranges, strides_arr)

        velocity = np.squeeze(velocity_data[f'velocity_{t:04d}'].values)
        temperature = np.squeeze(temperature_data[f'temperature_{t:04d}'].values)
        pressure = np.squeeze(pressure_data[f'pressure_{t:04d}'].values)

        for i in range(0, 2048 - patch_size + 1, stride):
            for j in range(0, 2048 - patch_size + 1, stride):
                v_patch = velocity[i:i+patch_size, j:j+patch_size, :]
                T_patch = temperature[i:i+patch_size, j:j+patch_size]
                p_patch = pressure[i:i+patch_size, j:j+patch_size]

                std_vort, std_T, std_P = compute_composite_metrics(v_patch, T_patch, p_patch)
                std_vort_list.append(std_vort)
                std_Tgrad_list.append(std_T)
                std_Pgrad_list.append(std_P)

    except Exception as e:
        print(f"❌ Failed for timestep {t}: {e}")

# -------------------------------
# Step 2: Compute thresholds
# -------------------------------
thresholds = {
    'vort': np.percentile(std_vort_list, 85),
    'temp': np.percentile(std_Tgrad_list, 85),
    'pres': np.percentile(std_Pgrad_list, 85),
}

print("\n📈 Thresholds computed from 85th percentile:")
print(thresholds)

# -------------------------------
# Step 3: Second Pass - Create dataset
# -------------------------------
X, y = [], []
print("\n📦 Generating dataset with new labels...")

def label_patch_composite(std_vort, std_T, std_P, thresholds):
    return int(std_vort > thresholds['vort'] or std_T > thresholds['temp'] or std_P > thresholds['pres'])

for t in tqdm(timesteps):
    try:
        axes_ranges = np.array([[0, 2047], [0, 2047], [16, 16]], dtype=np.int32)
        strides_arr = np.array([1, 1, 1], dtype=np.int32)

        velocity_data = getCutout(dataset, 'velocity', t, axes_ranges, strides_arr)
        temperature_data = getCutout(dataset, 'temperature', t, axes_ranges, strides_arr)
        pressure_data = getCutout(dataset, 'pressure', t, axes_ranges, strides_arr)

        velocity = np.squeeze(velocity_data[f'velocity_{t:04d}'].values)
        temperature = np.squeeze(temperature_data[f'temperature_{t:04d}'].values)
        pressure = np.squeeze(pressure_data[f'pressure_{t:04d}'].values)

        for i in range(0, 2048 - patch_size + 1, stride):
            for j in range(0, 2048 - patch_size + 1, stride):
                v_patch = velocity[i:i+patch_size, j:j+patch_size, :]
                T_patch = temperature[i:i+patch_size, j:j+patch_size]
                p_patch = pressure[i:i+patch_size, j:j+patch_size]

                std_vort, std_T, std_P = compute_composite_metrics(v_patch, T_patch, p_patch)
                label = label_patch_composite(std_vort, std_T, std_P, thresholds)

                input_tensor = np.stack([
                    v_patch[..., 0],
                    v_patch[..., 1],
                    T_patch
                ], axis=0)

                X.append(input_tensor)
                y.append(label)

    except Exception as e:
        print(f"❌ Failed for timestep {t}: {e}")

# -------------------------------
# Step 4: Save Dataset
# -------------------------------
X_tensor = torch.tensor(np.array(X), dtype=torch.float32)
y_tensor = torch.tensor(np.array(y), dtype=torch.long)

dataset_dict = {'X': X_tensor, 'y': y_tensor}
save_path = 'turbulence_dataset_sabl2048high_composite_label.pt'
torch.save(dataset_dict, save_path)

print(f"\n✅ Saved dataset to '{save_path}'")
print(f"Total samples: {len(y_tensor)} | 🌪️ Turbulence: {y_tensor.sum().item()} | 🛫 Non-turbulence: {(y_tensor == 0).sum().item()}")


🔍 Collecting statistics for thresholding...


  0% 0/9 [00:00<?, ?it/s]


-----
getCutout is processing...

total time elapsed = 37.279 seconds (0.621 minutes)

query completed successfully.
-----

-----
getCutout is processing...

total time elapsed = 10.848 seconds (0.181 minutes)

query completed successfully.
-----

-----
getCutout is processing...

total time elapsed = 12.419 seconds (0.207 minutes)

query completed successfully.
-----


 11% 1/9 [01:05<08:42, 65.32s/it]


-----
getCutout is processing...

total time elapsed = 32.538 seconds (0.542 minutes)

query completed successfully.
-----

-----
getCutout is processing...

total time elapsed = 12.538 seconds (0.209 minutes)

query completed successfully.
-----

-----
getCutout is processing...

total time elapsed = 10.448 seconds (0.174 minutes)

query completed successfully.
-----


...


-----
getCutout is processing...

total time elapsed = 17.741 seconds (0.296 minutes)

query completed successfully.
-----

-----
getCutout is processing...

total time elapsed = 3.967 seconds (0.066 minutes)

query completed successfully.
-----

-----
getCutout is processing...

total time elapsed = 3.948 seconds (0.066 minutes)

query completed successfully.
-----


 89% 8/9 [05:30<00:38, 38.33s/it]


-----
getCutout is processing...

total time elapsed = 12.686 seconds (0.211 minutes)

query completed successfully.
-----

-----
getCutout is processing...

total time elapsed = 2.941 seconds (0.049 minutes)

query completed successfully.
-----

-----
getCutout is processing...

total time elapsed = 3.068 seconds (0.051 minutes)

query completed successfully.
-----


100% 9/9 [05:54<00:00, 39.35s/it]



✅ Saved dataset to 'turbulence_dataset_sabl2048high_composite_label.pt'
Total samples: 36864 | 🌪️ Turbulence: 11268 | 🛫 Non-turbulence: 25596
